## Интерактивная визуализация

Модуль Dash Core Components (`dash.dcc`) включает в себя компонент `dcc.Graph`.

`dcc.Graph` отображает интерактивные визуализации данных с использованием открытого исходного кода plotly.js. Plotly.js поддерживает более 35 типов диаграмм и отображает диаграммы как в векторном формате SVG, так и в высокопроизводительном WebGL.

Компоненты Dash описываются набором атрибутов. Любой из этих атрибутов может быть обновлен с помощью функций обратного вызова, но только подмножество этих атрибутов обновляется при взаимодействии с пользователем, например при вводе текста внутри `dcc.Input` или по клику на опции в компоненте `dcc.Dropdown`.

Компонент dcc.Graph имеет четыре атрибута, которые могут изменяться при взаимодействии с пользователем: 
* hoverData; 
* clickData; 
* selectedData; 
* relayoutData. 

Эти свойства обновляются при наведении курсора мыши на точки, щелчке по точкам или выборе областей точек на графике.


In [8]:
%%file 04_graph/data.py
import json

from dash import Dash, dcc, html
from dash.dependencies import Input, Output
import plotly.express as px
import pandas as pd


app = Dash(
    __name__, 
    external_stylesheets=['https://codepen.io/chriddyp/pen/bWLwgP.css']
)

df = pd.DataFrame({
    "x": [1,2,1,2],
    "y": [1,2,3,4],
    "customdata": [1,2,3,4],
    "fruit": ["apple", "apple", "orange", "orange"]
})
styles = {
    'pre': {
        'border': 'thin lightgrey solid',
        'overflowX': 'scroll'
    }
}

fig = px.scatter(df, x="x", y="y", color="fruit", custom_data=["customdata"])
fig.update_layout(clickmode='event+select') # позволяет выделить точки с зажатым shift
fig.update_traces(marker_size=20)

app.layout = html.Div([
    dcc.Graph(
        id='basic-interactions',
        figure=fig
    ),
    html.Div(
        [
            html.Div(html.Pre(id='hover-data', style=styles['pre']), className='three columns'),
            html.Div(html.Pre(id='click-data', style=styles['pre']), className='three columns'),
            html.Div(html.Pre(id='selected-data', style=styles['pre']), className='three columns'),
            html.Div(html.Pre(id='relayout-data', style=styles['pre']), className='three columns'),
        ], 
        className='Row'
    )

])

@app.callback(
    Output('hover-data', 'children'),
    Input('basic-interactions', 'hoverData'))
# срабатывает при наведении курсора на точку
def display_hover_data(hoverData):
    return json.dumps(hoverData, indent=2)

@app.callback(
    Output('click-data', 'children'),
    Input('basic-interactions', 'clickData'))
# срабатывает при клике на точку
def display_click_data(clickData):
    return json.dumps(clickData, indent=2)


@app.callback(
    Output('selected-data', 'children'),
    Input('basic-interactions', 'selectedData'))
# срабатывает при выделении множества точек
def display_selected_data(selectedData):
    return json.dumps(selectedData, indent=2)


@app.callback(
    Output('relayout-data', 'children'),
    Input('basic-interactions', 'relayoutData'))
# срабатывает при масштабировании или нажатии на легенду
def display_relayout_data(relayoutData):
    return json.dumps(relayoutData, indent=2)



if __name__ == '__main__':
    app.run_server(debug=True)

Overwriting 04_graph/data.py


## Пример 2. Интерактивная визуализация графа

In [1]:
import numpy as np
import networkx as nx
import plotly.graph_objects as go

def to_scatter(G, pos):
    edge_x = []
    edge_y = []
    for edge in G.edges():
        x0, y0 = pos[edge[0]]
        x1, y1 = pos[edge[1]]
        edge_x.append(x0)
        edge_x.append(x1)
        edge_y.append(y0)
        edge_y.append(y1)
        edge_x.append(None)
        edge_y.append(None)
    edges_lines = np.c_[edge_x, edge_y]
    
    node_x = []
    node_y = []
    for node in G.nodes():
        x, y = pos[node]
        node_x.append(x)
        node_y.append(y) 
    nodes_markers = np.c_[node_x, node_y]

    node_text = np.array([f'Узел {n}' for n in G])
    
    fig = go.Figure(
        data = [
            go.Scatter(
                x=edges_lines[:, 0],
                y=edges_lines[:, 1],
                mode="lines",
                line_width=0.5,
                line_color="black",
                name="Связи"
            ),
            go.Scatter(
                x=nodes_markers[:, 0],
                y=nodes_markers[:, 1],
                mode="markers",
                marker_size=20,
                marker_color="blue",
                name="Узлы",
                text=node_text,
                hoverinfo='text',
            ),
        ],
    )
    return fig
    
G = nx.karate_club_graph()
pos = nx.spring_layout(G, seed=42)
fig = to_scatter(G, pos)
fig

In [13]:
%%file 04_graph/interactive.py

import networkx as nx
import plotly.graph_objects as go
from dash import Dash, dcc, html
from dash.dependencies import Input, Output
import plotly.express as px
import pandas as pd


app = Dash(
    __name__, 
    external_stylesheets=['https://codepen.io/chriddyp/pen/bWLwgP.css']
)

import numpy as np
def to_scatter(G, pos):
    edge_x = []
    edge_y = []
    for edge in G.edges():
        x0, y0 = pos[edge[0]]
        x1, y1 = pos[edge[1]]
        edge_x.append(x0)
        edge_x.append(x1)
        edge_y.append(y0)
        edge_y.append(y1)
        edge_x.append(None)
        edge_y.append(None)
    edges_lines = np.c_[edge_x, edge_y]
    
    node_x = []
    node_y = []
    for node in G.nodes():
        x, y = pos[node]
        node_x.append(x)
        node_y.append(y) 
    nodes_markers = np.c_[node_x, node_y]

    node_text = np.array([f'Узел {n}' for n in G])
    
    fig = go.Figure(
        data = [
            go.Scatter(
                x=edges_lines[:, 0],
                y=edges_lines[:, 1],
                mode="lines",
                line_width=0.5,
                line_color="black",
                name="Связи"
            ),
            go.Scatter(
                x=nodes_markers[:, 0],
                y=nodes_markers[:, 1],
                mode="markers",
                marker_size=20,
                marker_color="blue",
                name="Узлы",
                text=node_text,
                customdata = list(G),
                hoverinfo='text',
            ),
        ],
    )
    fig.update_layout(clickmode='event+select')
    return fig
 

G = nx.karate_club_graph()
pos = nx.spring_layout(G, seed=42)
fig1 = to_scatter(G, pos)
fig2 = to_scatter(nx.ego_graph(G, 0), pos)

app.layout = html.Div(
    [
        html.Div([dcc.Graph(id="full-graph", figure=fig1)], className="six columns"),
        html.Div([dcc.Graph(id="ego-graph", figure=fig2)], className="six columns")
    ],
    className='Row'
)

@app.callback(
    Output('ego-graph', 'figure'),
    Input('full-graph', 'clickData'))
# срабатывает при клике на точку
def display_ego(clickData):
    print(clickData)
    if clickData is None:
        return to_scatter(G, pos)
    sG = nx.compose_all([nx.ego_graph(G, point["customdata"]) for point in clickData["points"]] )

    return to_scatter(sG, pos)

    

if __name__ == '__main__':
    app.run_server(debug=True)

Overwriting 04_graph/interactive.py
